In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# TAA Post Processing

## Output Checking

### Standard Capacity Analysis Run with Default Initial Conditions

In [ ]:
#demandtrends folder
root="/home/craig/runs/big_test/base-testdata-v7/"
dtrends = root+ "DemandTrends.txt"
df=pd.read_csv(dtrends, sep='\t')
df.head()

In order to plot a line chart of TotalRequired and Deployed, we group by time and sum the values so that we have the total TotalRequired and Deployed for each day.  If you don't reset_index, you get a multi-index dataframe from groupby, which you can't plot, but functions called on groupby (like sum() here) will sum the values in each group.

In [ ]:
group_df = df.groupby(['t']).sum().reset_index()
group_df.head()

In [ ]:
plt.plot('t', 'TotalRequired', data=group_df)
plt.plot('t', 'Deployed', data=group_df)

### Random Initial Conditions Output Checks

We've been storing the results the the parent directory alongside the MARATHON workbook.  results.txt is from random initial condition runs from marathon.analysis.random.

In [ ]:
results = root+ "../results.txt" 
results

In [ ]:
df=pd.read_csv(results, sep='\t')
df.head()

Here we count the number records for each \[SRC, AC\] group.  For x initial condition reps and y phases, we should have x*y records.  This is essentially pivoting in Python by count.

In [ ]:
group_df = df.groupby(by=['SRC', 'AC']).count().reset_index()
group_df.head()

Check for any \[SRC, AC\] tuple that doesn't have x*y records.

In [ ]:
group_df[group_df['rep-seed']!=12]

## Post Processing

We'd like to compute Score and Excess for each \[SRC, AC\] tuple.  

First, average NG fill, then average RC fill, then average NG fill, then sum and divide by demand for Score (note that fill is fill from demandtrends and NOT just deployed like the field was renamed in 2327)
Excess is sum of available for each component divided by demand

In [ ]:
import numpy as np
#compute % demand met (dmet) and % excess over the demand (emet) 
#first by phase (use average group by with src, ac, phase)
def by_phase_percentages(results_df):
    group_df = results_df.groupby(by=['SRC', 'AC', 'phase']).mean().reset_index()
    group_df['dmet'] = np.where((group_df['total-quantity']==0), 1, 
                                                                (group_df['NG-fill'] + 
                                                                group_df['AC-fill'] + 
                                                                group_df['RC-fill']) / group_df['total-quantity'])
    #blaaah. what do I do?, this should be max for the phase instead.
    group_df['emet'] = np.where((group_df['total-quantity']==0), 1, 
                                                        (group_df['NG-deployable'] + 
                                                        group_df['AC-deployable'] + 
                                                        group_df['RC-deployable']) / group_df['total-quantity'])
    print(group_df['total-quantity'].isnull().sum())
    #this will be 0 because if there is no demand, we don't have a record.
    group_df.head()
    return group_df

Do first: 1 workbook
	(need to groupby.mean.unstack phase, but what do I expect?)
	Tab 1: src, ac, results by phase for demand 1, add score, excess
	Tab 2: src, ac, results by phase in columns for demand 2, add score excess
	Tab 3: src, ac, score-demand1, excess-demand1, score-dmd2, excess-dmd2, min-demand, min score.

In [ ]:
def merge_by_SRC_AC(table_list):
    return reduce(lambda  left,right: pd.merge(left,right,on=['SRC', 'AC'], 
                                            how='inner'), table_list)
def merge_by_SRC_AC2(table_list):
    return reduce(lambda  left,right: left.join(right,on=['SRC', 'AC'],
                                            how='inner'), table_list)

#just a place holder in the output table, was empty '' before
score_phase = 'combined'
def merge_two(left, right):
    #need to make the right table have a phase level in order to join it on right
    #we also want to name the Score and Excess phases combined so that our
    #our LibreCalc filters work properly.
    tuples = [('SRC', ''), ('AC', ''), ('Score', score_phase), ('Excess', score_phase)]
    right.columns=pd.MultiIndex.from_tuples(tuples, names=(None, 'phase'))
    return left.join(right.set_index(['SRC', 'AC']), how='inner')

def merge_by_SRC_AC3(table_list):
    return reduce(merge_two, table_list)
                                                
def results_by_phase(results_df):
    return results_df.groupby(by=['SRC', 'AC', 'phase']).mean().unstack(level=['phase'])

columns_by_phase=results_by_phase(df)
columns_by_phase.head()

In [ ]:
from functools import reduce 

#Weights used for a weighted score.
phase_weights= {"comp1" : 0.125,
               "comp2" : 0.125,
               "phase1" : .0625,
               "phase2" : .0625,
               "phase3" : .5,
               "phase4" : .125}

#add the weight to each row
def row_weight(row):
    return phase_weights[row['phase']]

#then group by src, ac, using custom function for weighted phases
def weighted_average(df, data_col, weight_col, by_col):
    df['weight']=df.apply(lambda row: row_weight(row), axis=1)
    df['_data_times_weight'] = df[data_col] * df[weight_col]
    g = df.groupby(by=by_col)
    #note that if we're missing a phase, the weight is adjusted accordingly
    #change: assume weight is 1 for all.
    res = g['_data_times_weight'].sum()  #/ g[weight_col].sum()
    print(res)
    del df['_data_times_weight']
    return res

#compute score and excess from demand met and demand excess
def by_src_inventory_scores(percentages_df):
    inventory_score = weighted_average(percentages_df, 'dmet', 'weight', ['SRC', 'AC']).to_frame()
    inventory_excess = weighted_average(percentages_df, 'emet', 'weight', ['SRC', 'AC']).to_frame()
    print(inventory_score.reset_index())
    res_df=inventory_score.reset_index().rename(columns={inventory_score.columns[0] : "Score"})
    print(res_df)
    #join both Score and Excess results.
    res_df = res_df.merge(inventory_excess.reset_index().rename(columns={inventory_excess.columns[0] : "Excess"}), 
                        how='inner', on=['SRC', 'AC'])
    return res_df

#compute score and excess from a path to results.txt
def by_src_ac_scores(results_path):
    df=pd.read_csv(results_path, sep='\t')
    results_df = results_by_phase(df)
    scores = by_src_inventory_scores(by_phase_percentages(df))
    return merge_by_SRC_AC3([results_df, scores])

by_src_ac_scores(results)

Now we should have a way to take a list of paths to results.txts and put them in one table, add a column called min_score, add another column called min_score_path to indicate the path with the min score

In [ ]:
import openpyxl

results_list = ["/home/craig/runs/big_test/results (copy 1).txt", "/home/craig/runs/big_test/results.txt"]
results_map = {'2.1' : "/home/craig/runs/big_test/results (copy 1).txt", 
               '7.1' : "/home/craig/runs/big_test/results.txt"}
score_list = map(lambda results: by_src_ac_scores(results), results_list) 
#need to check out this merge next...
'''df_merged = merge_by_SRC_AC(score_list)
df_merged=df_merged.rename(columns=
                                {'Score_x':'Score_7',
                                 'Excess_x':'Excess_7',
                                 'Score_y':'Score_2',
                                 'Excess_y':'Excess_2'})
df_merged.head()'''

writer = pd.ExcelWriter('TAA24-28_Modeling_Results.xlsx', engine='xlsxwriter')
left=pd.DataFrame()
for demand_name in results_map:
    scored_results = by_src_ac_scores(results_map[demand_name])
    print(scored_results)
    
    scores = scored_results[[('Score', score_phase), ('Excess', score_phase)]]
    scores.columns=['Score_'+demand_name, 'Excess_'+demand_name]
    if left.empty:
        left=scores
    else:
        right=scores
        left = pd.merge(left,right,on=['SRC', 'AC'], how='inner')
    scored_results.reset_index(inplace=True)
    #write to excel file here
    scored_results.to_excel(writer, sheet_name=demand_name)
left.head()  

#write third worksheet here
writer.save()

When writing the multi-index dataframes to Excel, pandas put an extra blank row below the column names, which messes up the filter in LibreOffice, but not Excel.  In Excel, you could turn the filter on the blank row.  In LibreOffice, that didn't work.  Although, in LibreOffice, you can turn it on the first row and it captures the first value.  Excel does not.  So those are the filter workarounds, but it looks cleaner to just remove that blank row.

In [ ]:
wb = openpyxl.reader.excel.load_workbook('TAA24-28_Modeling_Results.xlsx')
for demand_name in results_map:
    sh = wb.get_sheet_by_name(demand_name)
    sh.delete_rows(3, 1)
wb.save('TAA24-28_Modeling_Results.xlsx')

In [ ]:

#join tables so that you have two score columns
#add column called min_score
#add another column called min_score_demand
#could turn this into a map to concat both demand tables then, but not necessary

In [ ]:
#import xlsxwriter as writer


